In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Call Api on File Upload

Attributes
----------
subscriptions
events
notifications
luminesce
"""

toggle_code("Toggle Docstring")

# Call API on File Upload

This notebook will detail how to create a process in which actions are triggered on the upload of a new file.

For this example, we will manually drop an auto-load-transactions.xlsx file into [Drive](https://support.lusid.com/knowledgebase/article/KA-01672/en-us), and this will trigger an api call which will run a Luminesce query to create a portfolio, create instruments and upsert the transactions into that portfolio. This is a very simple example, but it could be extended further by triggering a job which could contain references to any number of apis.

The steps are as follows:
1. Authorise LUSID account
2. Create subscription to FileCreated event
3. Create webhook notification to be triggered by subscription
    - This notification will call an API
4. Trigger the event
    - This will cause the notification to fire which will call the API and in turn cause data from the uploaded file to be loaded into LUSID

Upon completion, the file is moved into a subfolder called archive, overwriting an existing file if one exists.

For further experimentation, the user can create entries with other portfolios.  The portfolio name is simply the upper case version of the portfolio code.

### Setup Lusid

Import modules and define our api factories.

In [2]:
import os
import json
import pandas as pd

# import lusid specific packages
import lusid.models as models
import lusid.utilities as utilities
import lusid
from fbnsdkutilities import ApiClientFactory
from lusidjam import RefreshingToken
import lusid_notifications as ln
import lusid_drive

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")
api_url = os.getenv("FBN_LUSID_API_URL")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

notifications_factory = ApiClientFactory(ln, token=RefreshingToken())

portfolios_factory = utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path
)

In [3]:
print ('LUSID Environment Initialised')
print ('LUSID API Version :', portfolios_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)


LUSID Environment Initialised
LUSID API Version : 0.6.11799.0


In [4]:
# notifications APIs
subscriptions_api = notifications_factory.build(ln.api.SubscriptionsApi)
notifications_api = notifications_factory.build(ln.api.NotificationsApi)
events_api = notifications_factory.build(ln.api.EventsApi)
event_types_api = notifications_factory.build(ln.api.EventTypesApi)

# portfolios API
portfolios_api = portfolios_factory.build(lusid.api.TransactionPortfoliosApi)

# drive API
configuration = lusid_drive.Configuration(
    host=f'{portfolios_api.api_client.configuration.host[:-4]}/drive'
)
configuration.access_token = portfolios_api.api_client.configuration.access_token

with lusid_drive.ApiClient(configuration) as api_client:
    files_api = lusid_drive.FilesApi(api_client)
    folders_api = lusid_drive.FoldersApi(api_client)

### 1. Authorise LUSID account

To authorise your LUSID account, follow the instruction at [KA-01735](https://support.lusid.com/knowledgebase/article/KA-01735/). This will allow the webhook to execute an API request on your behalf. You only need to do this once for each Lusid account.


### 2. Create a subscription

Here we are creating a subscription to a `FileCreated` event, with a matching filter set to look for our `auto-load-transactions.xlsx` file.

In [5]:
# create subscription to file creation event
def create_file_subscription(
        scope: str,
        code: str,
        name: str,
        matching_filter: str,
        description: str = None
):
    try:
        resp = subscriptions_api.create_subscription(
            create_subscription=ln.models.CreateSubscription(
                id=models.ResourceId(
                    scope=scope,
                    code=code
                ),
                display_name=name,
                description=description if description else name,
                status="Active",
                matching_pattern=ln.models.MatchingPattern(
                    event_type="FileCreated",
                    filter=matching_filter
                )
            )
        )
        print(f"Subscription created for {code}.")
        return resp
    except ln.ApiException as e:
        detail = json.loads(e.body)
        if detail["code"] != 711: # 'SubscriptionAlreadyExists'
            raise e
        else:
            print("Subscription already exists.")

create_file_subscription(
    scope="file_load",
    code="file_load",
    name="file_load",
    matching_filter="FileName eq 'auto-load-transactions.xlsx'",
)

Subscription created for file_load.


{'created_at': datetime.datetime(2023, 8, 28, 19, 43, 41, 265817, tzinfo=tzlocal()),
 'created_by': '00ujs4xwtp6rMTbuO2p7',
 'description': 'file_load',
 'display_name': 'file_load',
 'id': {'code': 'file_load', 'scope': 'file_load'},
 'last_modified_at': datetime.datetime(2023, 8, 28, 19, 43, 41, 265817, tzinfo=tzlocal()),
 'last_modified_by': '00ujs4xwtp6rMTbuO2p7',
 'matching_pattern': {'event_type': 'FileCreated',
                      'filter': "FileName eq 'auto-load-transactions.xlsx'"},
 'notifications': None,
 'status': 'Active',
 'use_as_auth': '00ujs4xwtp6rMTbuO2p7'}

### 3. Create a webhook notification

First, define the SQL Luminesce query to run. This query will unpack our data file from the drive and then create a portfolio, instruments and transactions.


In [6]:
sql = """
-- Extract transaction data from LUSID Drive

@txn_data =
use Drive.Excel
--file=/lusid-examples/auto-load-transactions.xlsx
enduse;

-- Set variables for the portfolio's scope and code

@@portfolio_scope = select 'auto-load-demo';
@portfolios = select distinct(portfolioCode),upper(portfolioCode) as PortfolioName from @txn_data;

-- Define the portfolio data

@create_portfolios =
select 'Transaction' as PortfolioType,
@@portfolio_scope as PortfolioScope,
portfolioCode as PortfolioCode,
PortfolioName as DisplayName,
'' as Description,
#2000-01-01# as Created,
''as SubHoldingKeys,
'GBP' as BaseCurrency
from @portfolios;

-- Upload the portfolio into LUSID

@response_create_portfolios =
select *
from Lusid.Portfolio.Writer
where ToWrite = @create_portfolios;

-- Get instrument data

@equity_instruments =
select
Name as DisplayName,
ISIN as Isin,
ClientInternal as ClientInternal,
SEDOL as Sedol,
'GBP' as DomCcy
from @txn_data;

-- Upload the transformed data into LUSID

@response = select *
from Lusid.Instrument.Equity.Writer
where ToWrite = @equity_instruments;

-- --Transform data using SQL

@transactions =
select
@@portfolio_scope as PortfolioScope,
t.PortfolioCode as PortfolioCode,
t.TransactionID as TxnId,
t.Type as Type,
t.TransactionDate as TransactionDate,
t.SettlementDate as SettlementDate,
t.Units as Units,
t.Price as TradePrice,
t.TotalConsideration as TotalConsideration,
t.Currency as SettlementCurrency,
t.ClientInternal as ClientInternal,
r.LusidInstrumentId as LusidInstrumentId
from @txn_data t
inner join @response r
where t.ClientInternal = r.ClientInternal;

-- Upload the transformed data into LUSID

select *
from Lusid.Portfolio.Txn.Writer
where ToWrite = @transactions;

-- Move the file into the archive directory

@data = select '/lusid-examples/auto-load-transactions.xlsx' as FullPath,
'/lusid-examples/archive/auto-load-transactions.xlsx' as NewFullPath,
'MoveRenameMayOverwrite' as Operation;

select * from Drive.File.Operation
where UpdatesToPerform = @data; 
"""

Now, define the webhook notification to run our SQL query.

In [7]:
def create_webhook_notification(
        scope: str,
        code: str,
        sql: str,
):
    # this can be changed to call any api required
    create_webhook_notification = {
        "description": "Upsert transactions into new portfolio",
        "httpMethod": "Put",
        "url": "/honeycomb/api/Sql/csv",
        "authenticationType": "Lusid",
        "contentType": "PlainText",
        "content": sql
    }

    try:
        response = notifications_api.create_webhook_notification(
            scope=scope,
            code=code,
            create_webhook_notification=create_webhook_notification
        )
        return f"Notification successfully created with ID : {response.id}"
    except ln.ApiException as e:
        return e.body

# delete notifications to allow updates
notifications = notifications_api.list_notifications(scope="file_load", code="file_load").values
for notification in notifications:
    notifications_api.delete_notification(
        scope="file_load",
        code="file_load",
        id=notification.id
    )

create_webhook_notification(
    scope="file_load",  # scope and code of subscription
    code="file_load",
    sql=sql
)

'Notification successfully created with ID : c20211be-501e-4dd8-8d81-c150c2428b30'

### 4. Trigger the event

You can trigger the event by dropping the `auto-load-transactions.xlsx` file into drive. This file is located within the data folder. You can verify the process has been successful by checking your portfolios for `uk-equity` within the `auto-load-demo` scope.

Alternatively you can run the following cells.

The following cell verifies that the portfolio doesn't exist

In [8]:
try:
    portfolios_api.get_transactions(scope='auto-load-demo', code='uk-equity').values
except lusid.ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 109: # 'PortfolioNotFound'
        raise e
    else:
        print(detail['title'])

Next, upload the file to drive. This will create a `FileCreated` event meaning that our event subscription will get triggered. This will kick off our webhook notification which will in turn call the Luminesce SQL API endpoint, with our SQL script as its payload. This SQL will run and begin the creation of our portfolio, instruments and transactions.

In [9]:
file_name = 'auto-load-transactions.xlsx'
drive_path = 'lusid-examples'

# create a directory
try:
    directory_response = folders_api.create_folder(
        create_folder={
            "path":'/',
            "name":drive_path
        }
    )
except lusid_drive.ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 664: # 'FolderAlreadyExists'
        raise e

with open('./data/auto-load-transactions.xlsx', 'rb') as data:
    try:
        response = files_api.create_file(
            x_lusid_drive_filename=file_name,
            x_lusid_drive_path=drive_path,
            content_length=os.stat('./data/auto-load-transactions.xlsx').st_size,
            body=data.read()
        )
        href = f"https://{portfolios_api.api_client.configuration.host[:-4]}/app/data-management/drive/{response.id}?type=file"
        print("File created at:", href)
    except lusid_drive.ApiException as e:
        detail = json.loads(e.body)
        if detail["code"] != 671: # 'FileAlreadyExists'
            raise e

File created at: https://https://fbn-dasaro.lusid.com/app/data-management/drive/248050f0-0488-4b20-9fe3-78a333c651cd?type=file


Now check that the portfolio exists and contains the expected transactions, for the portfolio defined in the initial file.

In [10]:
try:
    values = portfolios_api.get_transactions(scope='auto-load-demo', code='uk-equity').values
    portfolio_txns = pd.DataFrame(
        {
            'transaction_id': [i.transaction_id for i in values],
            'type': [i.type for i in values],
            'units': [i.units for i in values],
            'price': [i.transaction_price.price for i in values],
            'amount': [i.total_consideration.amount for i in values],
            'transaction_date': [i.transaction_date for i in values],
            'settlement_date': [i.settlement_date for i in values],
            'transaction_currency': [i.transaction_currency for i in values],
        }
    )
    portfolio_txns.head()
except lusid.ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 109: # 'PortfolioNotFound'
        raise e

In [11]:
portfolio_txns.sort_values(by="transaction_id").head(6)

,transaction_id,type,units,price,amount,transaction_date,settlement_date,transaction_currency
2,txn-0001,Buy,100.0,161.02,16102.0,2022-03-01 00:00:00+00:00,2022-03-04 00:00:00+00:00,GBP
4,txn-0002,Buy,100.0,1453.20,145320.0,2022-05-03 00:00:00+00:00,2022-05-07 00:00:00+00:00,GBP
1,txn-0003,Buy,100.0,45.31,4531.0,2022-02-04 00:00:00+00:00,2022-02-06 00:00:00+00:00,GBP
5,txn-0004,Buy,100.0,286.40,28640.0,2022-05-03 00:00:00+00:00,2022-05-06 00:00:00+00:00,GBP
3,txn-0005,Sell,100.0,1136.50,113650.0,2022-03-03 00:00:00+00:00,2022-03-05 00:00:00+00:00,GBP
0,txn-0006,Buy,100.0,2203.79,220379.0,2022-01-13 00:00:00+00:00,2022-01-16 00:00:00+00:00,GBP
